# ICP
In this tutorial we will lern to alignment of several point clouds using variants of the ICP algorithm.

We begin with loading the required modules.

In [ ]:
import numpy as np

from pyoints import (
	storage,
	nptools,
	Extent,
	transformation,
	filters,
	Coords,
)

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

%matplotlib inline

We load the standford bunny data.

In [ ]:
r = 0.004
A = storage.loadPly('bun000.ply')
T_A = transformation.r_matrix([90*np.pi/180, 0, 0])
A = A[list(filters.ball(A.indexKD(), r))]
A.transform(T_A)

B = storage.loadPly('bun045.ply')
T_B = transformation.r_matrix([90*np.pi/180, 0, 45*np.pi/180])
B = B[list(filters.ball(B.indexKD(), r))]
B.transform(T_B)

C = storage.loadPly('bun090.ply')
T_C = transformation.r_matrix([90*np.pi/180, 0, 90*np.pi/180])
C = C[list(filters.ball(C.indexKD(), r))]
C.transform(T_C)

print(A.shape)
print(A.dtype.descr)

Before we visuallize the point cloud, we define the axis limits.

In [ ]:
axis_lims = Extent([
	A.extent().center - 0.5 * A.extent().ranges.max(),
	A.extent().center + 0.5 * A.extent().ranges.max()
])

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection='3d')
ax.set_xlim(axis_lims[0], axis_lims[3])
ax.set_ylim(axis_lims[1], axis_lims[4])
ax.set_zlim(axis_lims[2], axis_lims[5])
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')

ax.scatter(*A.coords.T, color='green')
ax.scatter(*B.coords.T, color='blue')
ax.scatter(*C.coords.T, color='red')
plt.show()